In [1]:
import casadi as ca
# from MheSingleShooting import MheSingleShooting

# load data files
# PLEASE MODIFY THE PATH OF DATA!!!!!!!!!!!!!!!!!!!!!!!!!
# ===========================================================================================
# VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV
# # select data
data_subfold = "data_2023_03_01-16_48_29"    # the data from Lukas
# data_subfold = "rosbag2_2023_06_05-16_57_44"
# data_subfold = "rosbag2_2023_06_05-17_22_10"
# data_subfold = "rosbag2_2023_06_05-17_26_41"

# data_subfold = "rosbag2_2023_06_16-16_50_38"    # data for correction of imu coordinate 

# merge_command = False
save_merged_data = False

execfile("data_process_from_rosbags.py")
# AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
# ===========================================================================================

# Packages
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.models import DataRange1d
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

'data.hamster2__command.csv' loaded to 'df_command'
'data.hamster2__imu.csv' loaded to 'df_imu'
'data.hamster2__velocity.csv' loaded to 'df_velocity'
'data.tf.csv' loaded to 'df_tf'

Defined Variables:
    df_command
    df_imu
    df_velocity
    df_tf
    df_merge    # merged from 'df_command', 'df_velocity', 'df_imu' and 'df_tf'

Finish!


Loading BokehJS ...

## Load Sim Result

In [ ]:
# df_sim = pd.read_csv('./sim_result/sim_mhe_single.csv')
# df_sim = pd.read_csv('./sim_result/sim_mhe_multiple_tan.csv')
# df_sim = pd.read_csv('./sim_result/sim_mhe_multiple_sin.csv')
# df_sim = pd.read_csv('./sim_result/sim_mhe_multiple_cos.csv')
# df_sim = pd.read_csv('./sim_result/sim_mhe_multiple_correctedimu.csv')
# df_sim = pd.read_csv('./sim_result/MHE.csv')
# df_sim = pd.read_csv('./sim_result/MHE_halfway.csv')
df_sim = pd.read_csv('./sim_result/KF.csv')
# df_sim = pd.read_csv('./sim_result/KF_halfway.csv')

t0 = df_sim['time'][0]

df_sim.drop('Unnamed: 0', axis=1, inplace=True)

df_tf_sim = df_sim[df_sim['sensor.type']=='tf']   # drop the first row

df_diftf = df_tf_sim[1:]
df_difco = df_sim.loc[df_diftf.index-1]

df_diftf.reset_index(drop=True, inplace=True)
df_difco.reset_index(drop=True, inplace=True)

print(len(df_diftf))
print(len(df_difco))

df_sim

In [ ]:
dx = df_diftf['tf.x']-df_difco['x']
dy = df_diftf['tf.y']-df_difco['y']
dt = df_diftf['tf.yaw_z']-df_difco['theta']

# dx.plot.kde()
# dy.plot.kde()
# dt.plot.kde()
dx = dx.to_numpy()
dy = dy.to_numpy()
dt = dt.to_numpy()
print(sum(abs(dx))+sum(abs(dy))+sum(abs(dt)))
# print(min(dx), max(dx))
# print(min(dy), max(dy))
# print(min(dt), max(dt))

In [ ]:
dx = df_diftf['tf.x']-df_difco['x']
dy = df_diftf['tf.y']-df_difco['y']
dt = df_diftf['tf.yaw_z']-df_difco['theta']

# dx.plot.kde()
# dy.plot.kde()
# dt.plot.kde()
dx = dx.to_numpy()
dy = dy.to_numpy()
dt = dt.to_numpy()
print(sum(abs(dx))+sum(abs(dy))+sum(abs(dt)))
# print(min(dx), max(dx))
# print(min(dy), max(dy))
# print(min(dt), max(dt))

In [ ]:
df_sim['time']/1000

In [ ]:
# distribution
df_tf_sim['w_v'].plot.kde()
print(max(df_tf_sim['w_v']))

## Offline

In [ ]:
# ============================================ Plot =============================================
p_total = []

# simulation curve
p = plot_config(width=600, height=600, title='curve', x_label='x [ unknown ]', y_label='y [ unknown ]')
p.match_aspect=True
p.aspect_scale=1
p.triangle(df_sim['x'].iloc[0], df_sim['y'].iloc[0], legend_label='start_sim', fill_color="black", line_color='black', size=20)
p.circle(x=df_sim['x'], y=df_sim['y'], legend_label='sim_curve', fill_color="royalblue", line_color='royalblue', size=4)
p.asterisk(df_sim['x'].iloc[-1], df_sim['y'].iloc[-1], line_color="black", legend_label='end_sim', size=20)

# tf curve
p.triangle(df_tf_sim['tf.x'].iloc[0], df_tf_sim["tf.y"].iloc[0], legend_label='start_tf', fill_color="red", line_color='red', size=20)
p.circle(x=df_sim["tf.x"], y=df_sim["tf.y"], legend_label='tf_curve', line_color='orange', fill_color='orange', size=4)
p.circle(x=df_tf_sim["x"], y=df_tf_sim["y"], legend_label='sim_corrected_tf', line_color='black', fill_color='black', size=4)
p.asterisk(df_tf_sim["tf.x"].iloc[-1], df_tf_sim["tf.y"].iloc[-1], line_color="red", legend_label='end_tf', size=20)
p_total.append(p)

# simulation states
t0 = df_command["__time"][0]  # regard the global time stamp of first data point in "command" as 0s

p01 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p01.circle(x=df_sim['time'] - t0, y=df_sim['x'], legend_label='x', size=3, fill_color='green',line_color='green')
p01.circle(x=df_sim['time'] - t0, y=df_sim['tf.x'], legend_label='tf_x', size=2, fill_color='orange',line_color='orange')
p_total.append(p01)

p02 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p02.circle(x=df_sim['time'] - t0, y=df_sim['y'], legend_label='y', size=3, fill_color='black',line_color='black')
p02.circle(x=df_sim['time'] - t0, y=df_sim['tf.y'], legend_label='tf_y', size=2, fill_color='orange',line_color='orange')
p_total.append(p02)

p03 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p03.circle(x=df_sim['time'] - t0, y=df_sim['theta'], legend_label='theta', size=3, fill_color='royalblue',line_color='royalblue')
p03.circle(x=df_sim['time'] - t0, y=df_sim['tf.yaw_z'], legend_label='tf_theta', size=2, fill_color='orange',line_color='orange')
p_total.append(p03)

p04 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p04.circle(x=df_sim['time'] - t0, y=df_sim['v'], legend_label='v', size=3, fill_color='crimson',line_color='crimson')
p04.circle(x=df_sim['time'] - t0, y=df_sim['velocity'], legend_label='vel sensor', size=3, fill_color='orange',line_color='orange')

p_total.append(p04)

# # computational time
# p1 = plot_config(width=600, height=400, title='computational time', x_label='t [ s ]', y_label='computational time [ s ] ')
# p1.circle(x=df_sim['time'][2:] - t0, y=df_sim['t_obj'][2:]+df_sim['t_nlp'][2:], legend_label='plus', size=4, fill_color='black',line_color='black')
# p1.circle(x=df_sim['time'][2:] - t0, y=df_sim['t_obj'][2:], legend_label='t_obj', size=4, fill_color='royalblue',line_color='royalblue')
# p1.circle(x=df_sim['time'][2:] - t0, y=df_sim['t_nlp'][2:], legend_label='t_nlp', size=4, fill_color='red',line_color='red')
# p1.y_range = DataRange1d(start=0, end=0.05)
# p_total.append(p1)

# # diference between tf and before corrected
# p2 = plot_config(width=600, height=400, title='diference between tf and before corrected', x_label='t [ s ]', y_label='computational time [ s ] ')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.x']-df_tf_sim_minus1['x'], legend_label='x', size=4, fill_color='black',line_color='black')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.y']-df_tf_sim_minus1['y'], legend_label='y', size=4, fill_color='royalblue',line_color='royalblue')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.yaw_z']-df_tf_sim_minus1['theta'], legend_label='theta', size=4, fill_color='red',line_color='red')
# p_total.append(p2)


for i in p_total:
    show(i, notebook_handle=True)

# or
# from bokeh.io import output_file
# output_file(filename=f"./sim_KF_with_coordinate_correction.html", title="KF_with_coordinate_correction")
# output_file(filename=f"./sim_MHE_multiple.html", title="MHE_multiple")
# output_file(filename=f"./sim_MHE_single.html", title="MHE_single")
# output_file(filename=f"./sim_MHE_multiple_tan.html", title="MHE_multiple_tan")
# output_file(filename=f"./sim_MHE_multiple_sin.html", title="MHE_multiple_sin")
# output_file(filename=f"./sim_MHE_multiple_cos.html", title="MHE_multiple_cos")
# output_file(filename=f"./sim_MHE_multiple_correctedimu.html", title="MHE_multiple_correctedimu")
# show(gridplot(p_total, ncols=3))


## Real Time

In [ ]:
# ============================================ Plot =============================================
p_total = []

# # simulation curve
p = plot_config(width=600, height=600, title='curve', x_label='x [ unknown ]', y_label='y [ unknown ]')
p.match_aspect=True
p.aspect_scale=1
# rt_x
p.triangle(df_sim['rt_x'].iloc[0], df_sim['rt_y'].iloc[0], legend_label='start_sim', fill_color="black", line_color='black', size=20)
p.circle(x=df_sim['rt_x'], y=df_sim['rt_y'], legend_label='sim_curve', fill_color="royalblue", line_color='royalblue', size=4)
p.asterisk(df_sim['rt_x'].iloc[-1], df_sim['rt_y'].iloc[-1], line_color="black", legend_label='end_sim', size=20)
# x
# p.triangle(df_sim['x'].iloc[0], df_sim['y'].iloc[0], legend_label='start_sim', fill_color="black", line_color='black', size=20)
# p.circle(x=df_sim['x'], y=df_sim['y'], legend_label='sim_curve', fill_color="royalblue", line_color='royalblue', size=4)
# p.asterisk(df_sim['x'].iloc[-1], df_sim['y'].iloc[-1], line_color="black", legend_label='end_sim', size=20)


# # tf curve
p.triangle(df_tf_sim['tf.x'].iloc[0], df_tf_sim["tf.y"].iloc[0], legend_label='start_tf', fill_color="red", line_color='red', size=20)
p.circle(x=df_sim["tf.x"], y=df_sim["tf.y"], legend_label='tf_curve', line_color='orange', fill_color='orange', size=4)
p.circle(x=df_tf_sim["x"], y=df_tf_sim["y"], legend_label='sim_corrected_tf', line_color='black', fill_color='black', size=4)
p.asterisk(df_tf_sim["tf.x"].iloc[-1], df_tf_sim["tf.y"].iloc[-1], line_color="red", legend_label='end_tf', size=20)
p_total.append(p)

# p.triangle(df_tf['tf.x'].iloc[0], df_tf["tf.y"].iloc[0], legend_label='start_tf', fill_color="red", line_color='red', size=20)
# p.circle(x=df_tf["tf.x"], y=df_tf["tf.y"], legend_label='tf_curve', line_color='orange', fill_color='orange', size=4)
# p.circle(x=df_tf_sim["x"], y=df_tf_sim["y"], legend_label='sim_corrected_tf', line_color='black', fill_color='black', size=4)
# p.asterisk(df_tf["tf.x"].iloc[-1], df_tf["tf.y"].iloc[-1], line_color="red", legend_label='end_tf', size=20)
# p_total.append(p)

# # simulation states
t0 = df_command["__time"][0]  # regard the global time stamp of first data point in "command" as 0s
# rt_x
p01 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p01.circle(x=df_sim['time'] - t0, y=df_sim['rt_x'], legend_label='x', size=3, fill_color='green',line_color='green')
p01.circle(x=df_sim['time'] - t0, y=df_sim['tf.x'], legend_label='tf_x', size=2, fill_color='orange',line_color='orange')
p_total.append(p01)
p02 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p02.circle(x=df_sim['time'] - t0, y=df_sim['rt_y'], legend_label='y', size=3, fill_color='black',line_color='black')
p02.circle(x=df_sim['time'] - t0, y=df_sim['tf.y'], legend_label='tf_y', size=2, fill_color='orange',line_color='orange')
p_total.append(p02)
p03 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p03.circle(x=df_sim['time'] - t0, y=df_sim['rt_theta'], legend_label='theta', size=3, fill_color='royalblue',line_color='royalblue')
p03.circle(x=df_sim['time'] - t0, y=df_sim['tf.yaw_z'], legend_label='tf_theta', size=2, fill_color='orange',line_color='orange')
p_total.append(p03)
p04 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
p04.circle(x=df_sim['time'] - t0, y=df_sim['rt_v'], legend_label='v', size=3, fill_color='crimson',line_color='crimson')
p04.circle(x=df_sim['time'] - t0, y=df_sim['velocity'], legend_label='vel sensor', size=3, fill_color='orange',line_color='orange')
# x
# p01 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
# p01.circle(x=df_sim['time'] - t0, y=df_sim['x'], legend_label='x', size=3, fill_color='green',line_color='green')
# p01.circle(x=df_sim['time'] - t0, y=df_sim['tf.x'], legend_label='tf_x', size=2, fill_color='orange',line_color='orange')
# p_total.append(p01)
# p02 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
# p02.circle(x=df_sim['time'] - t0, y=df_sim['y'], legend_label='y', size=3, fill_color='black',line_color='black')
# p02.circle(x=df_sim['time'] - t0, y=df_sim['tf.y'], legend_label='tf_y', size=2, fill_color='orange',line_color='orange')
# p_total.append(p02)
# p03 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
# p03.circle(x=df_sim['time'] - t0, y=df_sim['theta'], legend_label='theta', size=3, fill_color='royalblue',line_color='royalblue')
# p03.circle(x=df_sim['time'] - t0, y=df_sim['tf.yaw_z'], legend_label='tf_theta', size=2, fill_color='orange',line_color='orange')
# p_total.append(p03)
# p04 = plot_config(width=600, height=400, title='system states', x_label='t [ s ]', y_label='states [ unknown ] ')
# p04.circle(x=df_sim['time'] - t0, y=df_sim['v'], legend_label='v', size=3, fill_color='crimson',line_color='crimson')
# p04.circle(x=df_sim['time'] - t0, y=df_sim['velocity'], legend_label='vel sensor', size=3, fill_color='orange',line_color='orange')

p_total.append(p04)

# # computational time
p1 = plot_config(width=600, height=400, title='computational time', x_label='t [ s ]', y_label='computational time [ s ] ')
p1.circle(x=df_sim['time'][0:] - t0, y=df_sim['t_com'][0:], legend_label='t_com', size=4, fill_color='royalblue',line_color='royalblue')
# p1.y_range = DataRange1d(start=0, end=max(df_sim['t_com']))
p_total.append(p1)

# # diference between tf and before corrected
# p2 = plot_config(width=600, height=400, title='diference between tf and before corrected', x_label='t [ s ]', y_label='computational time [ s ] ')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.x']-df_tf_sim_minus1['x'], legend_label='x', size=4, fill_color='black',line_color='black')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.y']-df_tf_sim_minus1['y'], legend_label='y', size=4, fill_color='royalblue',line_color='royalblue')
# p2.circle(x=df_tf_sim['time'] - t0, y=df_tf_sim['tf.yaw_z']-df_tf_sim_minus1['theta'], legend_label='theta', size=4, fill_color='red',line_color='red')
# p_total.append(p2)


for i in p_total:
    show(i, notebook_handle=True)

# or
# from bokeh.io import output_file
# output_file(filename=f"./sim_KF_realtime.html", title="KF_realtime")
# output_file(filename=f"./sim_KF_realtime_halfway.html", title="KF_realtime_halfway")

# show(gridplot(p_total, ncols=3))


In [ ]:
from casadi import *

# Declare variables
x = SX.sym("x", 2,1)
y = SX.sym("y", 2,1)
p = SX.sym("p", 2,1)

# Form the NLP
f = (x[0] - p[0]) ** 2 + (x[1] - p[1]) ** 2 + (y[0] - p[0]) ** 2 + (y[1] - p[1]) ** 2  # objective
# f = x[0] ** 2 + x[1] ** 2 + y[0] **2 + y[1]** 2  # objective
g = x[0] + x[1] -6  # constraint

# nlp = {'x': x, 'f': f, 'g': g}
nlp = {'x': vertcat(x,y), 'f': f, 'g': g, 'p': p} # p: parameter
# p = x+y
# nlp = {'x': vertcat(x,y), 'f': f, 'p': p}

# Pick an NLP solver
MySolver = "ipopt"
# MySolver = "worhp"
# MySolver = "sqpmethod"

# Solver options
opts = {}
if MySolver == "sqpmethod":
    opts["qpsol"] = "qpoases"
    opts["qpsol_options"] = {"printLevel": "none"}

# Allocate a solver
solver = nlpsol("solver", MySolver, nlp, opts)

# Solve the NLP
x0 = DM([0,0,0,0])
p0 = DM([2,-2])
sol = solver(x0=x0, p=p0, lbg=0)
# sol = solver(x0=x0, p=p0)

# Print solution
print("-----")
print("objective at solution = ", sol["f"])
print("primal solution = ", sol["x"])
print("dual solution (x) = ", sol["lam_x"])
print("dual solution (g) = ", sol["lam_g"])

In [ ]:
sol

In [ ]:
gpr_x.kernel_